In [1]:
import os
import numpy as np
import pandas as pd
import math
import collections
import filecmp
import xml.etree.ElementTree as ET
import random

In [36]:
#transit data check
#GTFS shared from Dan
#Source
#http://metro.kingcounty.gov/gtfs/
#https://developers.google.com/transit/gtfs/reference

path = 'G:/My Drive/2020/sumo/Transit_data_KCM/'
os.chdir(path)
transit_folder = os.listdir()
transit_file = []

for file in transit_folder:
    if 'zip' not in file:    
        file = file
        transit_file.append(file)

del(transit_folder)

for f in transit_file:
    if 'daily' in f:
        daily_transit = os.listdir(f)
    else:
        transit = os.listdir(f)

#check if two folders content are the same
if collections.Counter(daily_transit) == collections.Counter(transit):
    print('list content is the same')

#check if two txts are the same
for i in transit:
    print(filecmp.cmp(transit_file[0]+'/'+i,transit_file[1]+'/'+i, shallow=True))


In [51]:
#Check route file and generate pedestrian data
#scan of the .net.xml, find the pedestrian lane
path = 'G:/My Drive/2020/sumo/Ped/Peds/'
os.chdir(path)

files = os.listdir()
net_file = []
for f in files:
    if '.net' in f:
        net_file.append(f)

print('example.net.xml\n')
with open(net_file[0]) as file:
    for line in (file.readlines() [0:100]): 
        print(line)
        
print('Seattle.net.xml\n')
with open(net_file[1]) as file:
    for line in (file.readlines() [0:100]): 
        print(line)

example.net.xml

<?xml version="1.0" encoding="UTF-8"?>



<!-- generated on 03/09/20 21:20:52 by Eclipse SUMO netedit Version v1_5_0+0284-0846b7a875

<configuration xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/netconvertConfiguration.xsd">



    <input>

        <sumo-net-file value="E:\Fan\Dropbox\Personal_Work\Fan_Study&amp;Work\Abdullah\Ped\example.net.xml"/>

    </input>



    <output>

        <output-file value="E:\Fan\Dropbox\Personal_Work\Fan_Study&amp;Work\Abdullah\Ped\example.net.xml"/>

    </output>



    <processing>

        <geometry.min-radius.fix.railways value="false"/>

        <geometry.max-grade.fix value="false"/>

        <offset.disable-normalization value="true"/>

        <lefthand value="false"/>

    </processing>



    <junctions>

        <no-internal-links value="false"/>

        <no-turnarounds value="true"/>

        <junctions.corner-detail value="5"/>

        <junctions.limit-turn-

In [26]:
#example
#compare rou.xml and net.xml
#convert xml to csv
tree = ET.parse(net_file[0])
root = tree.getroot()
#root.findall('fcd-export')
lane_id_lst = []
for edge in root.findall('edge'):
    if edge.findall('edge')!=None:          
        if edge.get('priority') == "-1":
            for lane in edge.findall('lane'):
                if lane.get('allow') == "pedestrian":
                    ids = edge.get('id')
                    if ids not in lane_id_lst:
                        lane_id_lst.append(ids)
                        

tree = ET.parse(files[2])
root = tree.getroot()
#root.findall('fcd-export')
lanes = []
for person in root.findall('person'):
    for walk in person.findall('walk'):
        if walk.get('from') not in lanes:
            lanes.append(walk.get('from'))
        if walk.get('to') not in lanes:
            lanes.append(walk.get('to'))            

In [35]:
#get edge ids for Seattle network
def get_edge_id(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    #root.findall('fcd-export')
    lane_id_lst = []
    for edge in root.findall('edge'):
        if edge.findall('edge')!=None and edge.get('function')==None:              
            for lane in edge.findall('lane'):
                allow_string = lane.get('allow')
                if allow_string != None:
                    if 'pedestrian' in allow_string:
                        ids = edge.get('id')
                        if ids not in lane_id_lst:
                            lane_id_lst.append(ids)
    return(lane_id_lst)

#Seattle_edge_id = get_edge_id(net_file[1])

#get time list
def get_time_interval(od_lst,start_time, end_time):
    num_samples = len(od_lst[0])
    time_int = []
    for i in range(num_samples):
        radnum = random.randint(start_time, end_time)
        time_int.append(radnum)
    time_int = np.sort(time_int)

    time_lst = []
    for i in time_int:
        time_lst.append(str(i)+'.00')
        
    return(time_lst)

def get_od_lanes(edge_id_lst):
    od_lst = [[],[]]
    for i in edge_id_lst:
        for j in edge_id_lst:
            if i==j:
                pass
            else:
                od_lst[0].append(i)
                od_lst[1].append(j)
    return(od_lst)

#generate route
def write_rou_file(filename, od_lst, time_lst):
    f = open(filename, 'w')
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')
    
    num_samples = len(od_lst[0])
    for personid in range(num_samples):
        lines = '\t'+'<person id="ped'+str(personid)+'" depart="'+time_lst[personid]+'" departPos="random">\n'
        f.write(lines)
        #print(Seattle_edge_id[0])
        #print(Seattle_edge_id[personid+1])
        lines = '\t\t'+'<walk from="'+ od_lst[0][personid]+'" to="'+od_lst[1][personid]+'" arrivalPos="random"/>\n'
        f.write(lines)
        f.write('\t</person>\n')

    f.write('</routes>')

    f.close()
    return(1)

In [31]:
#test example
path = 'G:/My Drive/2020/sumo/Ped/sumo_test/'
os.chdir(path)
os.listdir()
test_edges_id = get_edge_id('crossing_test.net.xml')
od_lst = get_od_lanes(test_edges_id)
times=get_time_interval(od_lst, 0, 1799)

In [36]:
write_rou_file('crossing_rou.txt', od_lst, times)

1